In [ ]:
# @title Clone repository (hidden in Colab)
try:
    import google.colab

    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    !git clone https://github.com/tannhorn/some-notebooks.git
    %cd some-notebooks
    %cd kettle

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tannhorn/some-notebooks/blob/main/kettle/kettle.ipynb) [![GitHub](https://img.shields.io/badge/GitHub-black?logo=github)](https://github.com/tannhorn/some-notebooks/blob/main/kettle/kettle.ipynb)

First we import the necessary packages + the backend `data`.

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import plotly.graph_objects as go

import data

We set initial conditions and the time span for the solution.

In [ ]:
time_span: tuple[float, float] = (0, data.SIMULATION_TIME)  # Time span in seconds
initial_conditions: list[float] = [data.T_METAL_0, data.T_WATER_0]

We solve with constant heating.

In [ ]:
solution = solve_ivp(
    data.temperature_ode,
    time_span,
    initial_conditions,
    t_eval=np.linspace(0, data.SIMULATION_TIME, 1000),
    method="LSODA",
    args=(data.heating_function_constant,),
)

Here we extract results.

In [ ]:
# Extract results
time = solution.t
T_metal = solution.y[0]
T_water = solution.y[1]

_, fraction = data.boiling_heat_flux_vect(T_metal)

And plot.

In [ ]:
# Create figure
fig = go.Figure()

# Add Metal Temperature trace
fig.add_trace(
    go.Scatter(x=time, y=T_metal - 273.15, mode="lines", name="Metal Temperature")
)

# Add Water Temperature trace
fig.add_trace(
    go.Scatter(x=time, y=T_water - 273.15, mode="lines", name="Water Temperature")
)

# Add Saturation Temperature line
fig.add_trace(
    go.Scatter(
        x=[time[0], time[-1]],
        y=[data.T_SAT - 273.15] * 2,
        mode="lines",
        name="Saturation Temperature",
        line=dict(color="red", dash="dash"),
    )
)

# Add Boiling Intensity trace on secondary y-axis
fig.add_trace(
    go.Scatter(
        x=time,
        y=fraction/max(fraction),
        mode="lines",
        name="Boiling Intensity (right axis)",
        yaxis="y2",
    )
)

# Add vertical dashed line at the end of heating
fig.add_trace(
    go.Scatter(
        x=[data.TURNED_OFF_TIME, data.TURNED_OFF_TIME],
        y=[min(T_metal - 273.15), max(T_metal - 273.15)],
        mode="lines",
        name=f"Heating Stops @ {data.TURNED_OFF_TIME} s",
        line=dict(color="black", dash="dash", width=1),
    )
)

# Update layout
fig.update_layout(
    title="Constant Heating",
    xaxis=dict(title="Time (s)"),
    yaxis=dict(title="Temperature (°C)"),
    yaxis2=dict(title="Boiling Intensity (-)", overlaying="y", side="right"),
    legend=dict(x=0.5, y=0.1),
    template="plotly_white",
)

# Show the figure
fig.show()

Let's try now with a cycling function. First, let's plot the cycling function.

In [ ]:
values = [
    data.heating_function_cycle(t) / 1000
    for t in np.linspace(0, data.SIMULATION_TIME, 1000)
]  # in kW

# Create the Plotly figure
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=time,
        y=values,
        mode="lines",
        line_shape="hv",  # Use step-like appearance
    )
)

# Customize the layout
fig.update_layout(
    title=f"Heater power, cycle length {data.HEATING_CYCLE_LENGTH} s, turned off at {data.TURNED_OFF_TIME} s",
    xaxis_title="Time (s)",
    yaxis_title="Heater Power (kW)",
    template="plotly_white",
)

# Show the plot
fig.show()

Here we solve. Note the very small maximum step value, this is needed due to the step-like character of the heating function.

In [ ]:
solution = solve_ivp(
    data.temperature_ode,
    time_span,
    initial_conditions,
    t_eval=np.linspace(0, data.SIMULATION_TIME, 1000),
    method="LSODA",
    max_step=0.01,
    args=(data.heating_function_cycle,),
)

Let's extract results.

In [ ]:
# Extract results
time = solution.t
T_metal = solution.y[0]
T_water = solution.y[1]

_, fraction = data.boiling_heat_flux_vect(T_metal)

And plot.

In [ ]:
# Create figure
fig = go.Figure()

# Add Metal Temperature trace
fig.add_trace(
    go.Scatter(x=time, y=T_metal - 273.15, mode="lines", name="Metal Temperature")
)

# Add Water Temperature trace
fig.add_trace(
    go.Scatter(x=time, y=T_water - 273.15, mode="lines", name="Water Temperature")
)

# Add Saturation Temperature line
fig.add_trace(
    go.Scatter(
        x=[time[0], time[-1]],
        y=[data.T_SAT - 273.15] * 2,
        mode="lines",
        name="Saturation Temperature",
        line=dict(color="red", dash="dash"),
    )
)

# Add Boiling Intensity trace on secondary y-axis
fig.add_trace(
    go.Scatter(
        x=time,
        y=fraction/max(fraction),
        mode="lines",
        name="Boiling Intensity (right axis)",
        yaxis="y2",
    )
)

# Add vertical dashed line at the end of heating
fig.add_trace(
    go.Scatter(
        x=[data.TURNED_OFF_TIME, data.TURNED_OFF_TIME],
        y=[min(T_metal - 273.15), max(T_metal - 273.15)],
        mode="lines",
        name=f"Heating Stops @ {data.TURNED_OFF_TIME} s",
        line=dict(color="black", dash="dash", width=1),
    )
)

# Update layout
fig.update_layout(
    title="Cycling Heating",
    xaxis=dict(title="Time (s)"),
    yaxis=dict(title="Temperature (°C)"),
    yaxis2=dict(title="Boiling Intensity (-)", overlaying="y", side="right"),
    legend=dict(x=0.75, y=0.1),
    template="plotly_white",
)

# Show the figure
fig.show()